# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [4]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/Colab Notebooks/R10/Project2/"

### Loading the Glove Embeddings

In [0]:
#from zipfile import ZipFile
#with ZipFile(project_path+'glove.6B.zip', 'r') as z:
#  z.extractall(path=project_path)

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import numpy as np

In [0]:
tb = pd.read_csv('/content/drive/My Drive/Colab Notebooks/R10/Project2/train_bodies.csv')

In [0]:
ts = pd.read_csv('/content/drive/My Drive/Colab Notebooks/R10/Project2/train_stances.csv')

In [9]:
tb.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [10]:
ts.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
dataset = pd.merge(tb,ts,on='Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [12]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [60]:
finalText = []
for x in dataset.articleBody.unique():
  finalText.append(x)

for y in dataset.Headline.unique():
  finalText.append(y)

print(len(finalText))

3317


In [0]:
tokenizer.fit_on_texts(finalText)

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



In [62]:
print('word_counts' , tokenizer.word_counts)
print('document_count ', tokenizer.document_count)
print('word_index ' , tokenizer.word_index)
print('word_docs ' , tokenizer.word_docs)

word_counts OrderedDict([('a', 16009), ('small', 203), ('meteorite', 180), ('crashed', 8), ('into', 816), ('wooded', 19), ('area', 245), ('in', 13416), ("nicaragua's", 17), ('capital', 213), ('of', 14743), ('managua', 58), ('overnight', 25), ('the', 37341), ('government', 674), ('said', 4220), ('sunday', 195), ('residents', 105), ('reported', 713), ('hearing', 65), ('mysterious', 33), ('boom', 33), ('that', 7882), ('left', 390), ('16', 93), ('foot', 61), ('deep', 44), ('crater', 112), ('near', 372), ("city's", 20), ('airport', 198), ('associated', 127), ('press', 291), ('reports', 785), ('spokeswoman', 81), ('rosario', 16), ('murillo', 29), ('committee', 71), ('formed', 22), ('by', 3480), ('to', 16702), ('study', 43), ('event', 160), ('determined', 29), ('it', 4324), ('was', 6025), ('relatively', 23), ('appears', 196), ('have', 3521), ('come', 282), ('off', 549), ('an', 2882), ('asteroid', 103), ('passing', 53), ('close', 180), ('earth', 134), ('house', 374), ('sized', 21), ('2014', 40

### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
from nltk.tokenize import sent_tokenize 

In [63]:
texts = []
for ab in dataset.articleBody:
  texts.append(ab)
print(len(texts))

49972


In [64]:
articles = []
for tx in texts:
  articles.append(nltk.sent_tokenize(tx))
print(len(articles))

49972


## Check 2:

first element of texts and articles should be as given below. 

In [65]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [66]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence

In [0]:
word_counts = tokenizer.word_counts
document_counts = tokenizer.document_count
word_idx = tokenizer.word_index
word_docs = tokenizer.word_docs

idx_word = tokenizer.index_word

In [0]:
lenart = len(articles)
data = np.zeros((lenart,MAX_SENTS,MAX_SENT_LENGTH))

In [0]:
art_idx = 0
for art in articles:  
  sent_idx = 0
  for sent in art:
    word_idx = 0
    for word in text_to_word_sequence(sent):
      data[art_idx][sent_idx][word_idx] = tokenizer.word_index.get(word)
      word_idx += 1
      if(word_idx == MAX_SENT_LENGTH): break
    sent_idx += 1
    if(sent_idx == MAX_SENTS): break
  art_idx += 1

### Check 3:

Accessing first element in data should give something like given below.

In [72]:
data[0, :, :]

array([[3.0000e+00, 4.0400e+02, 4.5300e+02, 6.3210e+03, 8.2000e+01,
        3.0000e+00, 3.6170e+03, 3.1300e+02, 5.0000e+00, 3.9070e+03,
        3.7600e+02, 4.0000e+00, 1.5060e+03, 2.9310e+03, 1.0000e+00,
        1.0300e+02, 1.5000e+01, 4.2000e+02, 0.0000e+00, 0.0000e+00],
       [8.2600e+02, 9.4000e+01, 1.3300e+03, 3.0000e+00, 2.3870e+03,
        2.3880e+03, 7.0000e+00, 1.7400e+02, 3.0000e+00, 9.3100e+02,
        1.4230e+03, 1.8800e+03, 7.6500e+02, 1.9100e+02, 1.0000e+00,
        3.4830e+03, 4.1400e+02, 1.0000e+00, 6.6800e+02, 2.5300e+02],
       [1.0300e+02, 1.0840e+03, 4.0630e+03, 2.6190e+03, 1.5000e+01,
        3.0000e+00, 1.2240e+03, 3.2450e+03, 1.9000e+01, 1.0000e+00,
        1.0300e+02, 2.0000e+00, 1.9180e+03, 1.0000e+00, 5.3100e+02,
        2.6200e+03, 1.4000e+01, 1.0000e+01, 3.0000e+00, 3.1270e+03],
       [1.8900e+02, 3.3720e+03, 8.4200e+02, 1.6200e+02, 2.3360e+03,
        4.3000e+01, 5.5330e+03, 1.8370e+03, 1.0370e+03, 5.0000e+00,
        1.0570e+04, 1.6599e+04, 1.0000e+00, 6

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [73]:
texts_heading = []
for Hl in dataset.Headline:
  texts_heading.append(Hl)
print(len(texts_heading))

49972


In [74]:
articles_heading = []
for th in texts_heading:
  articles_heading.append(nltk.sent_tokenize(th))
print(len(articles_heading))

49972


In [75]:
texts_heading[0]

'Soldier shot, Parliament locked down after gunfire erupts at war memorial'

In [76]:
articles_heading[0]

['Soldier shot, Parliament locked down after gunfire erupts at war memorial']

In [0]:
data_heading = np.zeros((len(articles_heading), MAX_SENTS, MAX_SENT_LENGTH), dtype=int)

In [0]:
art_idx = 0
for art in articles_heading:  
  sent_idx = 0
  for sent in art:
    word_idx = 0
    for word in text_to_word_sequence(sent):
      data_heading[art_idx][sent_idx][word_idx] = tokenizer.word_index.get(word)
      word_idx += 1
      if(word_idx == MAX_SENT_LENGTH): break
    sent_idx += 1
    if(sent_idx == MAX_SENTS): break
  art_idx += 1

In [79]:
data_heading[0, :, :]

array([[  593,   233,   279,  4026,   169,    34,  1359, 12475,    21,
          230,   714,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0],
       [  

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [80]:
oneHotEncoding = pd.get_dummies(dataset.Stance)
oneHotEncoding.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


In [81]:
labels = np.array(oneHotEncoding)
print(labels)

[[0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]
 ...
 [1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]]


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [82]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [85]:
train_idx = round(data.shape[0]*0.8)
train_idx

39978

In [86]:
x_train = data[:train_idx]
x_val = data[train_idx:]
len(x_train)+len(x_val)

49972

In [87]:
x_heading_train = data_heading[:train_idx]
x_heading_val = data_heading[train_idx:]
len(x_heading_train)+len(x_heading_val)

49972

In [88]:
y_train = labels[:train_idx]
y_val = labels[train_idx:]
len(y_train)+len(y_val)

49972

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [89]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39978, 20, 20)
(39978, 4)
(9994, 20, 20)
(9994, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [93]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('/content/drive/My Drive/Colab Notebooks/R10/Project2/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
vocab_size = len(tokenizer.word_index)
embedding_matrix = np.zeros((vocab_size, 100))


for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

from keras.models import Model
from keras.layers import Input

### Model

In [95]:
data_inputs = Input(shape=(None, 20))
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(data_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [encoder_state_h, encoder_state_c]
encoder_model = Model(data_inputs, encoder_states)
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 20)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 283648    
Total params: 283,648
Trainable params: 283,648
Non-trainable params: 0
_________________________________________________________________


In [96]:
data_heading_inputs = Input(shape=(None, 20))
# We set up our decoder to not return full output sequences,
# and to return internal states. We don't use the
# return states in the training model.
decoder_lstm = LSTM(256, return_sequences=False, return_state=True)
decoder_outputs, _, _ = decoder_lstm(data_heading_inputs, initial_state=encoder_states)
decoder_dense = Dense(4, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)
model = Model([data_inputs, data_heading_inputs], decoder_outputs)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 20)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 283648      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 283648      input_2[0][0]                    
                                                                 lstm_1[0][1]               

### Compile and fit the model

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [98]:
model.fit([x_train, x_heading_train], y_train, batch_size=64, epochs=25)

Epoch 1/25
39978/39978 [==============================] - 127s 3ms/step - loss: 0.7892
Epoch 2/25
39978/39978 [==============================] - 128s 3ms/step - loss: 0.7380
Epoch 3/25
39978/39978 [==============================] - 128s 3ms/step - loss: 0.7084
Epoch 4/25
39978/39978 [==============================] - 129s 3ms/step - loss: 0.6856
Epoch 5/25
39978/39978 [==============================] - 125s 3ms/step - loss: 0.6620
Epoch 6/25
39978/39978 [==============================] - 127s 3ms/step - loss: 0.6449
Epoch 7/25
39978/39978 [==============================] - 126s 3ms/step - loss: 0.6245
Epoch 8/25
39978/39978 [==============================] - 124s 3ms/step - loss: 0.6114
Epoch 9/25
39978/39978 [==============================] - 129s 3ms/step - loss: 0.5945
Epoch 10/25
39978/39978 [==============================] - 125s 3ms/step - loss: 0.5790
Epoch 11/25
39978/39978 [==============================] - 125s 3ms/step - loss: 0.5744
Epoch 12/25
39978/39978 [================

In [0]:
y_pred = model.predict([x_val, x_heading_val])

In [100]:
score = model.evaluate([x_val, x_heading_val],y_val)
print(model.metrics_names, ":", score)

9994/9994 [==============================] - 12s 1ms/step
['loss'] : 0.5693269327941932


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)